In [1]:
from rsna_retro.imports import *
from rsna_retro.metadata import *
from rsna_retro.preprocess import *
from rsna_retro.train import *
# from rsna_retro.train3d import *
from rsna_retro.train_adjacent import *

Loading imports


In [2]:
torch.cuda.set_device(0)

## Train

In [3]:
name = 'train_adjacent_3c_stg1'

In [4]:
dls = get_adj_data(512, 128, Meta.splits_stg1)
learn = get_learner(dls, xresnet18, pretrained=True)

In [5]:
do_fit(learn, 12, 4e-2)
learn.save(f'runs/{name}-1')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.123005,0.128054,0.960500,0.918314,14:26
1,0.110744,0.133835,0.962090,0.924413,14:20
2,0.104014,0.113242,0.966272,0.931110,14:28
3,0.096419,0.121459,0.962801,0.917563,14:47
4,0.094224,0.118429,0.964254,0.928627,14:31
5,0.092598,0.102938,0.968336,0.936406,14:31
6,0.089526,0.103072,0.969818,0.938367,14:39
7,0.083365,0.089662,0.972118,0.943065,14:29
8,0.077166,0.088185,0.973043,0.945904,14:27
9,0.073223,0.085705,0.974189,0.947839,14:27


In [6]:
learn.load(f'runs/{name}-1')
learn.dls = get_adj_data(512, 256, Meta.splits_stg1)
do_fit(learn, 12, 4e-3)
learn.save(f'runs/{name}-2')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.079003,0.089902,0.972086,0.944210,15:47
1,0.075470,0.084672,0.973028,0.945471,15:32
2,0.073782,0.088990,0.972364,0.943230,15:30
3,0.070363,0.081974,0.975014,0.951174,15:32
4,0.068445,0.081242,0.974979,0.950716,15:34
5,0.067710,0.080689,0.975507,0.951251,15:35
6,0.064289,0.082163,0.975379,0.950500,15:32
7,0.061378,0.078406,0.976303,0.952842,15:35
8,0.058710,0.079316,0.976303,0.952728,15:32
9,0.055686,0.078987,0.976578,0.953046,15:33


In [7]:
learn.dls = get_adj_data(256, 384, Meta.splits_stg1, img_dir=path_jpg)
do_fit(learn, 4, 4e-4)
learn.save(f'runs/{name}-3')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.061176,0.075894,0.976387,0.954039,37:56
1,0.057947,0.074355,0.976920,0.954956,37:44
2,0.055271,0.074277,0.977272,0.955554,37:43
3,0.052953,0.074520,0.977215,0.955643,37:33


## Submission

In [5]:
learn.load(f'runs/{name}-3')
sub_fn = f'subm/{name}'

In [6]:
# learn.dls = get_data_gen(Meta.df_tst, bs=512, sz=None, with_aug=False, tst_dir='tst_jpg256')
learn.dls = get_adj_test_data(512, sz=384, tst_dir='tst_jpg')

In [7]:
preds,targs = learn.get_preds()

In [8]:
pred_csv = submission(Meta.df_tst, preds, fn=sub_fn)

In [9]:
api.competition_submit(f'{sub_fn}.csv', f'{name} adjacent 3c', 'rsna-intracranial-hemorrhage-detection')

100%|██████████| 26.0M/26.0M [00:03<00:00, 9.08MB/s]


Successfully submitted to RSNA Intracranial Hemorrhage Detection

In [12]:
api.competitions_submissions_list('rsna-intracranial-hemorrhage-detection')[0]

{'ref': 14276524,
 'totalBytes': 27277209,
 'date': '2020-01-23T19:42:49.467Z',
 'description': 'train_adjacent_3c_stg1 adjacent 3c',
 'errorDescription': None,
 'fileName': 'train_adjacent_3c_stg1.csv',
 'publicScore': '0.62870',
 'privateScore': '0.05836',
 'status': 'complete',
 'submittedBy': 'Andrew Shaw',
 'submittedByRef': 'bearpelican',
 'teamName': 'Andrew Shaw',
 'type': 'standard',
 'url': 'https://www.kaggle.com/submissions/14276524/14276524.raw'}